Transport phenomena is the study of the transferring of momentum, energy and mass. Oftentimes these processes are coupled as in boiling water, where bubbles of air trapped in the water nucleate due to the elevated temperature reducing the solubility of gases, followed by the density differences between the bubble and water causing the bubble to rise. These processes are further mediated through convection currents caused by temperature gradients in the boiling water. 

In this example, we will look at a far simple example of energy transport, characterized as the change in temperature of a heat source placed in an infinitely large cooler surrounding region. We will assume periodic boundary conditions to overcome the limitation of finite sizes in compute and assume that the temperature in the source reduces at the same time.  (INSERT IMAGE HERE)

# Theory

For our problem, we will be basing our grid on rectilinear or cartesian coordinates. First, we will review the relevant transport equations. A more detailed explanation of each expression and derivation can be found in BSLK. We begin by writing a general energy transport equation through a control volume.

$$ change = accumulation + removal + generation + destruction $$

In our closed system, there is no generation, destruction and accumulation of energy, meaning that the change in energy over time can be written as

$ change = removal$

If we approximate that the energy lost from the addition of heat is lost through conduction, the removal of energy can be approximated using fourier's law, defined as 

$$ q = -k \nabla T $$

where $k (\frac{W}{mK})$ is the thermal conductivity of the matrix, $\nabla T (\frac{K}{m})$ is the temperature gradient between the heated area and its surroundings and $q (\frac{W}{m^2 K})$ is the heat flux through a surface. This is related to the energy change at each grid point over time as, 

$\frac{d E}{dt} = \nabla q$

Where $E (\frac{J}{m^3})$ is the energy density of a specific grid point and $t (s)$ is the time. This leads to the expression, 

$$ \frac{d E}{d t} = -k \nabla^2 T $$

The energy density of the heat source can be expressed from the heat capacity of the heat source, defined as $E = \rho c_p (T - T_{ref})$. If we make the further approximation that density $\rho (\frac{kg}{m^3})$ and specific heat capacity $\frac{J}{K kg}$ stay constant with temperature, we can rearrange the above expression to obtain the following PDE written as a function of temperature exclusively,

$$ \frac{dT}{dt} = -\frac{k}{\rho c_p} \nabla^2 T$$

Oftentimes, the factor $\frac{k}{\rho c_p}$ is condensed into the term $\alpha (\frac{m^2}{s})$ defining the thermal diffusivity of the system and will be used as an input parameter.

Now that we have laid out the basic theory being implemented, we shall turn to how we can convert math to something a computer can solve. There are derivatives of time and space that we will need to solve for. I will stick to the simplest ones here as the focus is on creating an illustration how the code looks different between implementations in different languages. However, the literature of how both are done is vast with (SOURCES PROVIDED).

## Finite difference

The simplest way to discretize a differential equation in space is using a finite difference technique. The core idea of this scheme is we convert the continous differential equations above into discrete sums and differences which we can solve. Commonly, this is done using a central difference scheme shown below for the first $f^1_x$ and second derivative $f^2_x$ for a function $f(x,y,t)$

\begin{align}
f^1_x &= \frac{f(x + h, y, t) - f(x - h, y, t)}{2h} \\
f^2_x &= \frac{f(x + h, y, t) - 2f(x,y,t) + f(x - h, y, t)}{h^2} 
\end{align}

Where $h$ is the spacing between grids. Central difference offers produced second order error compared to forward or backward difference which produce first order error.

## Forward Euler

Next, I look at how we can integrate over time. Multiple time integration schemes exist which are split into explicit and implicit schemes. Explicit schemes include techniques such as Forward Euler and the Runge-Kutta time integrators while implicit schemes include the Backwards Euler and Crank-Nicholson schemes. For now, we will stick to the Forward Euler technique. 

Forward Euler solves an initial value problem by iterating a value of a variable through time using the initial value and the change in the variable through time. Notationally, this looks like

\begin{align}
g(x, y, t_0) &= b \\
\frac{dg}{dt} &= f(x,y,t) \\
g(x, y , t + \Delta t) &= g(x, y, t) + \Delta t \frac{dg}{dt}
\end{align}

where $g(x, y, t)$ is a value of a function at time $t$ and location $(x,y)$, $\frac{dg}{dt}$ is the derivative of function $g$ with respect to time $t$ and is expressed as any function $f(x,y,t)$. The value of $g(x, y, t + \Delta t)$ is a sum of the value of $g(x, y, t)$ and the timestep over which the derivative $\frac{dg}{dt}$ is calculated.

The Forward Euler technique can be numerically unstable. Therefore care must be taken when selecting parameters for time and space integration and must fall within the CFL condition, defined as $C = \frac{u \Delta t}{\Delta x} < 1$.

## Python implementation

Using the established rules above, we move to generating a prototype in python. We assume the heat source places is a circle at the middle of the system with a radius that can be specified with the system containing periodic boundary conditions. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def animate_plot(x, y, interval=50):
    """
    Create an animation of a 2D line plot over time.

    This function takes two lists of lists or numpy arrays representing the x and y 
    coordinates of data over multiple timesteps and generates an animated plot. The 
    animation can be rendered in a Jupyter notebook using the :class:`IPython.display.HTML` object.

    :param x: 
        The x-coordinates of the data, with shape (t, L), where t is the number 
        of timesteps and L is the length of data at each timestep.
    :type x: list of lists or numpy.ndarray
    :param y: 
        The y-coordinates of the data, with shape (t, L), where t is the number 
        of timesteps and L is the length of data at each timestep.
    :type y: list of lists or numpy.ndarray
    :param interval: 
        The delay in milliseconds between frames in the animation. Default is 50.
    :type interval: int, optional

    :return: 
        An animation object that can be rendered in a Jupyter notebook using 
        :class:`IPython.display.HTML`.
    :rtype: matplotlib.animation.FuncAnimation

    :examples:
        >>> import numpy as np
        >>> from IPython.display import HTML
        >>> x = [np.linspace(0, 2 * np.pi, 100)] * 50
        >>> y = [np.sin(xi + i * 0.1) for i, xi in enumerate(x)]
        >>> ani = animate_plot(x, y, interval=100)
        >>> HTML(ani.to_jshtml())
    """

    fig, ax = plt.subplots()
    ln, = plt.plot(x[0], y[0])

    def get_min_max(arr):
        arr_min = 0
        arr_max = 0
        for i in range(len(arr)):
            if min(arr[i]) < arr_min:
                arr_min = min(arr[i])
            if max(arr[i]) > arr_max:
                arr_max = max(arr[i])
        return [arr_min, arr_max]

    def init():
        plt.ylim(get_min_max(y))
        plt.xlim(get_min_max(x))
        ln.set_data(x[0], y[0])
        return ln,

    def update(i):
        ln.set_data(x[i], y[i])
        return ln,

    ani = animation.FuncAnimation(fig, update, frames=len(x), init_func=init, interval=interval, blit=True)
    plt.close()
    return ani

In [ ]:
def make_system(nx, ny, R, T_hot = 373, T_cold = 273):
    X, Y = np.meshgrid(*[np.arange(0, nx), np.arange(0, ny)])
    out = np.where((X - nx/2)**2 + (Y - ny/2)**2 <= R**2, T_hot, T_cold)
    return out

def FD_timestep(grid, dx, dy, dt, alpha, halo):
    new_grid = grid.copy()
    nx, ny = grid.shape # current shape includes the halo points on the edge of the grid

    slc = np.s_[halo:nx-halo, halo:ny-halo] # array slicing for non edge slices
    
    # central difference scheme for laplacian in x direction for spatial integration
    central_difference_x = (grid[halo+1:nx-halo+1, halo:ny-halo] - 2*grid[halo:nx-halo, halo:ny-halo] + grid[halo-1:nx-halo-1, halo:ny-halo])/dx**2
    # central difference scheme for laplacian in y direction for spatial integration
    central_difference_y = (grid[halo:nx-halo, halo+1:ny-halo+1] - 2*grid[halo:nx-halo, halo:nx-halo] + grid[halo:nx-halo, halo-1:nx-halo-1])/dy**2
    
    # performing forward euler for time integration for non halo regions
    new_grid[slc] = grid[slc] + alpha*dt*(central_difference_x + central_difference_y)

    return new_grid

def periodic_boundary(grid, halo):
    nx, ny = grid.shape # current shape includes the halo points on the edge of the grid
    new_grid = grid.copy()

    # periodic boundary for y direction copies the values in the other side of the 'real' region to the halo
    new_grid[halo:halo+halo,:] = grid[nx-halo*halo:nx-halo*halo+halo, :]
    new_grid[nx-halo*halo:nx-halo*halo+halo,:] = grid[halo:halo+halo, :]

    # periodic boundary for x direction copies the values in the other side of the 'real' region to the halo
    new_grid[:,halo:halo+halo] = grid[:,ny-halo*halo:ny-halo*halo+halo]
    new_grid[:,ny-halo*halo:ny-halo*halo+halo] = grid[:,halo:halo+halo]

    return new_grid

In [ ]:
nx = 200
ny = 200
halo = 2
R = 30

dx = 1/nx
dy = 1/ny
timesteps = 100
dt = 0.001
dump_freq = 100

alpha = 0.001
T_hot = 373
T_cold = 273
C = alpha/(min(dx, dy)**2/dt)

if C > 1:
    raise RuntimeWarning("CFL condition not fulfilled")

temperature_fields_time = np.empty((timesteps//dump_freq+1, nx, ny))

grid_old = make_system(nx+2*halo, ny+2*halo, R)
grid_new = np.empty((nx+2*halo, ny+2*halo))


for time in range(0, timesteps+1):
    if time%dump_freq == 0:
        temperature_fields_time[time//dump_freq] = grid_old[halo:nx+halo, halo:ny+halo]
    
    grid_new = FD_timestep(grid_old, 1, 1, dt, alpha, halo)
    grid_new = periodic_boundary(grid_new, halo)

    grid_old = grid_new.copy()

In [ ]:
plt.imshow(temperature_fields_time[-1])
plt.colorbar()

# Single core implementation in C++

In [ ]:
def read_data(file_path):
    data = []
    with open(file_path, "r") as f:
        list_of_text = f.readlines()
        for curr_line in list_of_text:
            curr_data = [float(i) for i in curr_line.split(",")[:-1]]
            data.append(curr_data)

    data = np.array(data)

    return data

# Multicore (CPU) implementation in C++

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob

In [ ]:
path = "src"
data_paths = sorted(glob.glob(f"{path}/T*.txt"))

In [ ]:
data = read_data(data_paths[-1])
plt.imshow(data)
plt.colorbar()